In [93]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [94]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [95]:
LOCAL = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [96]:
%%capture pwd
!pwd

In [97]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('how-might-scottish-independence-affect-the-costs-of-international-trade',
 300,
 500)

# Fig 1

In [98]:
configSource = "raw/F1_Share_of_rest_of_UK_in_Scottish_Trade.json"
config = json.loads(open(configSource, "r").read())
df = pd.read_csv(config["data"]["url"])

In [99]:
f = "fig1_trade"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,Year,Trade,Percentage
0,1998,Exports,59
1,1999,Exports,59
2,2000,Exports,59
3,2001,Exports,59
4,2002,Exports,61


In [100]:
df.columns

Index(['Year', 'Trade', 'Percentage'], dtype='object')

In [ ]:
base = (
    alt.Chart(f1)
    .encode(
        x=alt.X(
            "y:T",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                format="%Y",
            ),
        )
    )
    .transform_pivot("Trade", value="Percentage", groupby=["Year"])
    .transform_calculate(y="toDate(toString(datum.Year)+'-01-01')")
)
line1 = base.mark_line(color=colors["eco-turquiose"]).encode(
    y=alt.Y(
        "Exports:Q",
        axis=alt.Axis(
            grid=True,
            gridColor=colors["eco-gray"],
            gridOpacity=0.1,
            title="%",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=20,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=6,
            format=".0f",
        ),
        scale=alt.Scale(domain=[50, 80]),
    )
)
line2 = base.mark_line(color=colors["eco-mid-blue"]).encode(
    y=alt.Y(
        "Imports:Q",
    )
)
area = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.4,
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color=colors["eco-turquiose"], offset=0.1),
            alt.GradientStop(color="white", offset=0.5),
            alt.GradientStop(color=colors["eco-light-blue"], offset=0.9),
        ],
        x1=1,
        x2=1,
        y1=1,
        y2=0,
    ),
).encode(y2="Exports:Q", y="Imports:Q")
label1 = (
    alt.Chart(pd.DataFrame([{"x": "2006-01-01", "y": 71, "t": "Imports"}]))
    .mark_text(color=colors["eco-mid-blue"])
    .encode(x="x:T", y="y:Q", text="t:N")
)
label2 = (
    alt.Chart(pd.DataFrame([{"x": "2006-01-01", "y": 60.5, "t": "Exports"}]))
    .mark_text(color=colors["eco-turquiose"])
    .encode(x="x:T", y="y:Q", text="t:N")
)
layer1 = (
    ((line1 + area + line1 + line2 + label1 + label2).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "w").write(readme)
layer1

# Fig 2

In [ ]:
df = pd.read_excel("raw/data.xlsx", sheet_name="Figure 2")
df.columns = ["country", "ruk", "rscot"]

In [ ]:
f = "fig2_residual"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

In [ ]:
base = (
    alt.Chart(f2)
    .encode(
        x=alt.X(
            "rscot:Q",
            sort=[],
            axis=alt.Axis(
                grid=False,
                titleAlign="right",
                titleAnchor="end",
                title="Scotland residual trade",
                titleY=-15,
                titleX=402,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=10,
                orient="bottom",
                labelAngle=0,
            ),
        ),
        tooltip=["country:N", "Scotland:Q", "UK:Q"],
    )
    .transform_calculate(Scotland="round(datum.rscot*100)/100.0")
    .transform_calculate(UK="round(datum.ruk*100)/100.0")
)
scatter = base.mark_point(
    color=colors["eco-mid-blue"], fill=colors["eco-light-blue"], size=50
).encode(
    y=alt.Y(
        "ruk:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="Rest of UK residual trade",
            titleX=5,
            titleY=5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    )
)
axis1 = (
    alt.Chart(pd.DataFrame([{"y": 0}]))
    .mark_rule(strokeWidth=1, opacity=0.3, color=colors["eco-gray"])
    .encode(y=alt.Y("y:Q"))
)
axis2 = (
    alt.Chart(pd.DataFrame([{"x": 0}]))
    .mark_rule(strokeWidth=1, opacity=0.3, color=colors["eco-gray"])
    .encode(x=alt.X("x:Q"))
)
labels = scatter.mark_text(color=colors["eco-mid-blue"], dy=12, fontSize=10).encode(
    text="country:N"
)
layer1 = (
    ((axis1 + axis2 + scatter + labels).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1